In [1]:
from mcm_orh_functions import *
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import inspect


import warnings
warnings.filterwarnings("ignore")


orh_data = pd.read_csv('./orh_2022.csv')

orh_data = format_orh(orh_data)

orh_data.head()

,org,org_house_name,Stage,mother_first_i,father_first_i,fav_color,fav_season,age,out_of_state_to_RH,out_of_county_to_RH,...,recovery_housing_difference_from_other_living_environments,start_date,submission_date,network_id,tags,input_type,Gender,Sexuality,Race,CJS
0,12 step life,12 Step Life: The Ethel Hardy House,Follow Up,N,D,Green,Fall,18-24 years,No,No,...,NaN,12/9/22 4:50,12/9/22 4:55,4a9fc7602d,NaN,Client,Woman,Straight (Heterosexual),Black or African American,Not involved with the criminal justice system
1,12 step life,12 Step Life: The Ethel Hardy House,Follow Up,N,D,Green,Fall,18-24 years,No,No,...,NaN,11/21/22 3:00,11/21/22 3:06,4a9fc7602d,NaN,Client,Woman,Straight (Heterosexual),Black or African American,Not involved with the criminal justice system
2,12 step life,12 Step Life: The Ethel Hardy House,Follow Up,L,W,Red,Summer,60-64 years,No,No,...,NaN,9/23/22 1:03,9/23/22 1:08,4a9fc7602d,NaN,Client,Woman,Straight (Heterosexual),Black or African American,Not involved with the criminal justice system
3,12 step life,12 Step Life: The Ethel Hardy House,Follow Up,V,T,Blue,Summer,35-39 years,No,No,...,NaN,9/23/22 0:25,9/23/22 0:32,4a9fc7602d,NaN,Client,Woman,Straight (Heterosexual),White,Currently on parole/probation
4,12 Step Life/Ethel Hardy House,12 Step Life: The Ethel Hardy House,Follow Up,L,W,Blue,Fall,60-64 years,No,No,...,NaN,7/24/22 5:41,7/24/22 5:48,4a9fc7602d,NaN,Client,Woman,Straight (Heterosexual),Black or African American,Not involved with the criminal justice system


In [62]:
# Criminal Justice System
cjs = orh_data[((orh_data['curr_status_cjs_parole_probation'].notna())
               | (orh_data['curr_status_cjs_drug_court'].notna()))
               &(orh_data['curr_status_cjs_no_involvement'].isna())
               &(orh_data['curr_status_cjs_no_answer'].isna())]
no_cjs = orh_data[(orh_data['curr_status_cjs_no_involvement'] == 'Not involved with the criminal justice system')]
cjs_title = "In Criminal Justice System"
no_cjs_title = "Not In Criminal Justice System"


# Parents
parent = orh_data[(orh_data['parent'] == 'Yes')]
non_parent = orh_data[(orh_data['parent'] == 'No')]
parent_title = "Parent"
non_parent_title = "Not Parent"


# Male vs Female
female = orh_data[(orh_data['gender_identify_woman'] == 'Woman')
                  &(orh_data['gender_identify_man'] != 'Man')]
male = orh_data[(orh_data['gender_identify_man'] == 'Man')
                 &(orh_data['gender_identify_woman'] != 'Woman')]
female_title = "Female Identifying"
male_title = "Male Identifying"


# Heterosexual vs LGBTQ+
hetero = orh_data[orh_data['sexual_identity_heterosexual'].notna()]
lgbtq = orh_data[((orh_data['sexual_identity_asexual'].notna())
                  |(orh_data['sexual_identity_bisexual'].notna())
                  |(orh_data['sexual_identity_gay'].notna())
                  |(orh_data['sexual_identity_lesbian'].notna())
                  |(orh_data['sexual_identity_pansexual'].notna())
                  |(orh_data['sexual_identity_queer'].notna())
                  |(orh_data['sexual_identity_same_gender_loving'].notna())
                  |(orh_data['sexual_identity_other'].notna()))
                  &((orh_data['sexual_identity_heterosexual'].isna()))
                  &((orh_data['sexual_identity_no_answer'].isna()))]
hetero_title = "Heterosexual"
lgbtq_title = "LGBTQ+"


# Race
white = orh_data[(orh_data['race_id_white'].notna())
                 & (orh_data['race_id_black_or_african_american'].isna())
                 & (orh_data['race_id_american_indian_or_alaska_native'].isna())
                 & (orh_data['race_id_chinese'].isna())
                 & (orh_data['race_id_vietnamese'].isna())
                 & (orh_data['race_id_native_hawaiian'].isna())
                 & (orh_data['race_id_filipino'].isna())
                 & (orh_data['race_id_korean'].isna())
                 & (orh_data['race_id_samoan'].isna())
                 & (orh_data['race_id_asian_indian'].isna())
                 & (orh_data['race_id_japanese'].isna())
                 & (orh_data['race_id_chamorro'].isna())
                 & (orh_data['race_id_other_asian'].isna())
                 & (orh_data['race_id_other_pacific_islander'].isna())
                 & (orh_data['rad_id_no_answer'].isna())
                 & (orh_data['race_id_other'].isna())]
bipoc = orh_data[((orh_data['race_id_white'].isna())
                  & (orh_data['rad_id_no_answer'].isna()))
                  & ((orh_data['race_id_black_or_african_american'].isna())
                     | (orh_data['race_id_american_indian_or_alaska_native'].isna())
                     | (orh_data['race_id_chinese'].isna())
                     | (orh_data['race_id_vietnamese'].isna())
                     | (orh_data['race_id_native_hawaiian'].isna())
                     | (orh_data['race_id_filipino'].isna())
                     | (orh_data['race_id_korean'].isna())
                     | (orh_data['race_id_samoan'].isna())
                     | (orh_data['race_id_asian_indian'].isna())
                     | (orh_data['race_id_japanese'].isna())
                     | (orh_data['race_id_chamorro'].isna())
                     | (orh_data['race_id_other_asian'].isna())
                     | (orh_data['race_id_other_pacific_islander'].isna())
                     | (orh_data['race_id_other'].isna()))]
white_title = "White"
bipoc_title = "BIPOC"

# Staff vs Client Exit

client_exit = orh_data[(orh_data['Stage']=='Move In') |
         ((orh_data['Stage']=='Move Out')&(orh_data['input_type']=='Client'))]
staff_exit = orh_data[(orh_data['Stage']=='Move In') |
         ((orh_data['Stage']=='Move Out')&(orh_data['input_type']=='Staff'))]
client_title = 'Client Exit'
staff_title = 'Staff Exit'




In [63]:
# Criminal Justice History
outcomeComparison(cjs,
                  cjs_title,
                  no_cjs,
                  no_cjs_title,
                  plot=False)
cohortComparison(cjs,
                 cjs_title,
                 no_cjs,
                 no_cjs_title,
                 stage = "Move In",
                 plot=False)

# Parents
outcomeComparison(parent,
                  parent_title,
                  non_parent,
                  non_parent_title,
                  plot=False)
cohortComparison(parent,
                 parent_title,
                 non_parent,
                 non_parent_title,
                 stage = "Move In",
                 plot=False)

# Male vs Female
outcomeComparison(male,
                  male_title,
                  female,
                  female_title,
                  plot=False)
cohortComparison(male,
                 male_title,
                 female,
                 female_title,
                 stage = "Move In",
                 plot=False)

# Heterosexual vs LGBTQ+
outcomeComparison(hetero,
                  hetero_title,
                  lgbtq,
                  lgbtq_title,
                  plot=False)
cohortComparison(hetero,
                 hetero_title,
                 lgbtq,
                 lgbtq_title,
                 stage = "Move In",
                 plot=False)

# Race
outcomeComparison(white,
                  white_title,
                  bipoc,
                  bipoc_title,
                  plot=False)
cohortComparison(white,
                 white_title,
                 bipoc,
                 bipoc_title,
                 stage = "Move In",
                 plot=False)

# Staff vs Client Exit - Outcomes Only
outcomeComparison(client_exit,
                  client_title,
                  staff_exit,
                  staff_title,
                  plot=False)

Alcohol Use In Criminal Justice System: Sample Size: Stage
Move In     1182
Move Out     251
dtype: int64

Alcohol Use In Criminal Justice System: Summary Table
Stage                  Move In  Move Out
Alcohol Use                             
1-10 days             0.084602  0.059761
11-20 days            0.019459  0.003984
21-30 days            0.020305  0.011952
No use                0.868020  0.908367
Prefer not to answer  0.007614  0.015936


Drug Use In Criminal Justice System: Sample Size: Stage
Move In     1182
Move Out     251
dtype: int64

Drug Use In Criminal Justice System: Summary Table
Stage                  Move In  Move Out
Drug Use                                
1-10 days             0.089679  0.055777
11-20 days            0.048223  0.011952
21-30 days            0.040609  0.003984
No use                0.811337  0.900398
Prefer not to answer  0.010152  0.027888


Alcohol Use Not In Criminal Justice System: Sample Size: Stage
Move In     1124.0
Move Out     243.0
dtype